## Deliverable 2. Create a Customer Travel Destinations Map.

In [61]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,62.60,77,100,3.44,overcast clouds
1,1,Belaya Gora,RU,68.5333,146.4167,-12.62,90,100,7.85,overcast clouds
2,2,Notodden,NO,59.5594,9.2585,28.99,53,8,4.85,clear sky
3,3,Hilo,US,19.7297,-155.0900,73.99,78,90,11.50,light rain
4,4,Mataura,NZ,-46.1927,168.8643,55.00,63,92,13.00,overcast clouds


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
11,11,Sulangan,PH,11.1373,123.7228,80.26,78,83,16.51,broken clouds
14,14,Mar Del Plata,AR,-38.0023,-57.5575,78.80,47,0,8.05,clear sky
16,16,Vaini,TO,-21.2000,-175.2000,82.40,89,75,10.36,light rain
17,17,Salalah,OM,17.0151,54.0924,78.80,57,0,4.61,clear sky
18,18,Cabedelo,BR,-6.9811,-34.8339,84.20,74,20,11.50,few clouds
20,20,Rikitea,PF,-23.1203,-134.9692,80.53,79,100,19.89,moderate rain
23,23,Cayenne,GF,4.9333,-52.3333,82.40,83,75,12.66,broken clouds
26,26,Soure,BR,-0.7167,-48.5233,77.29,89,100,6.20,overcast clouds
30,30,Sibu,MY,2.3000,111.8167,75.20,100,40,0.67,scattered clouds
31,31,Faanui,PF,-16.4833,-151.7500,81.90,76,100,19.82,light rain


In [63]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()


City_ID                178
City                   178
Country                178
Lat                    178
Lng                    178
Max Temp               178
Humidity               178
Cloudiness             178
Wind Speed             178
Current Description    178
dtype: int64

In [73]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

#no empty rows in this DataFrame, but would use .dropna()
clean_df = preferred_cities_df.dropna()


In [74]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Sulangan,PH,80.26,broken clouds,11.1373,123.7228,
14,Mar Del Plata,AR,78.80,clear sky,-38.0023,-57.5575,
16,Vaini,TO,82.40,light rain,-21.2000,-175.2000,
17,Salalah,OM,78.80,clear sky,17.0151,54.0924,
18,Cabedelo,BR,84.20,few clouds,-6.9811,-34.8339,
20,Rikitea,PF,80.53,moderate rain,-23.1203,-134.9692,
23,Cayenne,GF,82.40,broken clouds,4.9333,-52.3333,
26,Soure,BR,77.29,overcast clouds,-0.7167,-48.5233,
30,Sibu,MY,75.20,scattered clouds,2.3000,111.8167,
31,Faanui,PF,81.90,light rain,-16.4833,-151.7500,


In [75]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
 
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [81]:
# 7. Drop the rows where there is no Hotel Name.
filter = hotel_df["Hotel Name"] != ""
clean_hotel_df = hotel_df[filter]

clean_hotel_df.count()


City                   162
Country                162
Max Temp               162
Current Description    162
Lat                    162
Lng                    162
Hotel Name             162
dtype: int64

In [83]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Sulangan,PH,80.26,broken clouds,11.1373,123.7228,Maia's Beach Resort
14,Mar Del Plata,AR,78.80,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
16,Vaini,TO,82.40,light rain,-21.2000,-175.2000,Keleti Beach Resort
17,Salalah,OM,78.80,clear sky,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
18,Cabedelo,BR,84.20,few clouds,-6.9811,-34.8339,Hotel Pigalle
20,Rikitea,PF,80.53,moderate rain,-23.1203,-134.9692,Pension Maro'i
23,Cayenne,GF,82.40,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi
26,Soure,BR,77.29,overcast clouds,-0.7167,-48.5233,Casarão
30,Sibu,MY,75.20,scattered clouds,2.3000,111.8167,RH HOTEL (4 Star)
31,Faanui,PF,81.90,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa


In [84]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [89]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [90]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))